In [1]:
#user_input = input('Input the comedy transcript you would like to predict: ') #use this to take in real-time input and run the pipeline
user_input = open('test_pipeline_transcript.txt', 'r').read()

In [2]:
user_input = ' '.join(user_input.split('\n'))

In [3]:
import numpy as np
import pandas as pd
from pipeline_funcs import *
from imblearn.over_sampling import RandomOverSampler
import pickle

In [4]:
valid_transcripts_sent_df = pd.read_pickle('chloe_valid_transcripts_sent_df')
valid_transcripts_sent_df

,Comedian,Title,Subtitle,Transcript Part,Action,Processed Transcript,Audience Reaction,Funniness
0,Chris Rock,Selective Outrage (2023) | Transcript,NaN,"She said, “$300, I’ll do anything you want.” ...",[audience laughing] [audience cheering],anything bitch paint house need death penalty ...,cheer laugh,2
1,Chris Rock,Selective Outrage (2023) | Transcript,NaN,Ladies and gentlemen. Ladies and gentlemen. C...,[audience cheering] [audience continue cheering],lady gentleman lady gentleman chris rock,cheer,1
2,Chris Rock,Selective Outrage (2023) | Transcript,NaN,"What’s up, Baltimore?",[audience cheers loudly],baltimore,cheer,1
3,Chris Rock,Selective Outrage (2023) | Transcript,NaN,"Yes! Yes, yes. Thank you! Thank you so much! ...",[audience cheering],yes yes yes thank thank much thank much netfli...,cheer,1
4,Chris Rock,Selective Outrage (2023) | Transcript,NaN,"I’mma try… N*gga, sit down! I’mma try to do a...",[audience laughing],mma try… n gga sit mma try show tonight withou...,laugh,2
...,...,...,...,...,...,...,...,...
6337,NaN,Jim Jefferies on Gun Control [Full Transcript],Australian comedian Jim Jefferies breaks down ...,"“Guns! Who wants to sell me a gun?” Now, I’m ...",[Audience cheering],gun sell gun wrap talk anymore now… agree bear...,cheer,1
6338,NaN,Jim Jefferies on Gun Control [Full Transcript],Australian comedian Jim Jefferies breaks down ...,Yeah! And that made a hell of a lot of sense ...,NA,hell lot sense musket government drone bring g...,NA,0
6339,Reggie Watts,Spatial (2016) – Full Transcript,Completely improvised show weaving together sk...,” Fuck. # ..Then jump for my love # Jump in # ...,NA,jump love jump feel touch jump taste kiss nigh...,NA,0
6340,GEORGE CARLIN,COMPLAINTS AND GRIEVANCES (2001) – FULL TRANSC...,"Full transcript of Complaints and Grievances, ...",” Now. Folks. This next piece of material’s go...,NA,folk next piece material give u chance bond am...,NA,0


In [5]:
valid_transcripts_df = pd.read_pickle('valid_transcripts_df')
valid_transcripts_df

,Unnamed: 0,Comedian,Date,Title,Subtitle,Transcript,Cleaned Transcript,Polarity,Subjectivity
0,0,Chris Rock,"March 8, 2023",Selective Outrage (2023) | Transcript,NaN,[slow instrumental music playing] [funk drums ...,anything bitch paint house need death penalty ...,0.068457,0.501896
1,1,Marc Maron,"March 3, 2023",Thinky Pain (2013) | Transcript,Marc Maron returns to his old stomping grounds...,[siren wailing] I don’t know what you were thi...,nervous choice small venue theater seem comedy...,0.038824,0.522475
6,6,Dave Chappelle,"January 22, 2023",What’s in a Name (2022) | Transcript,Dave Chappelle delivers a speech at his presti...,What’s in a Name? is a 40-minute talk Chappell...,name minute talk chappelle deliver duke elling...,0.125232,0.526631
7,7,Iliza Shlesinger,"December 20, 2022",Hot Forever (2022) | Transcript,With topics ranging from tight rompers to ugly...,[upbeat music playing] [crowd cheering] Clevel...,cleveland ohio thank thank much great nice pub...,0.043022,0.504384
8,8,Gabriel Iglesias,"November 28, 2022",Stadium Fluffy (2022) | Transcript,"Features Gabriel ""Fluffy"" Iglesias as he talks...",[man] Can you please state your name? Martin M...,please state name martin moreno might as… mart...,0.079741,0.484695
...,...,...,...,...,...,...,...,...,...
410,410,George Carlin,"April 6, 2017",Jamming in New York (1992) – Transcript,NaN,Jammin’ in New York is George Carlin’s 14th al...,jammin new york george carlin th album eighth ...,0.045662,0.492957
411,411,NaN,"April 1, 2017",Jim Jefferies on Gun Control [Full Transcript],Australian comedian Jim Jefferies breaks down ...,Australian comedian Jim Jefferies breaks down ...,australian comedian jim jefferies break absurd...,-0.004999,0.520927
412,412,Reggie Watts,"March 30, 2017",Spatial (2016) – Full Transcript,Completely improvised show weaving together sk...,"Hello, I’m Thomas. I’m so glad to meet you Mum...",hello thomas glad meet mum freddie meet girl t...,0.171253,0.573918
413,413,GEORGE CARLIN,"March 29, 2017",COMPLAINTS AND GRIEVANCES (2001) – FULL TRANSC...,"Full transcript of Complaints and Grievances, ...",Complaints and Grievances is a HBO stand-up sp...,complaint grievance hbo stand special comedian...,0.080087,0.470308


In [6]:
#filter transcripts in 'valid_transcripts_df' for those similar to 'user_input' in terms of polarity and subjectivity
filtered_transcripts_df = filter_by_sentiment(valid_transcripts_df, user_input)
filtered_transcripts_df

,Unnamed: 0,Comedian,Date,Title,Subtitle,Transcript,Cleaned Transcript,Polarity,Subjectivity
0,391,Patrice O’Neal,"April 28, 2017",Elephant in the Room (2011) – Transcript,"Transcript of 'Elephant in the Room', stand up...",[Light funk music] * [cheers and applause] Tha...,thank thank thank thank thank thank particular...,0.029343,0.532460
1,117,Jim Gaffigan,"January 19, 2020",Quality Time (2019) – Full Transcript,Comedian Jim Gaffigan performs live at the Sta...,"(audience cheering, applauding) Thank you! Tha...",audience cheer applaud thank thank god thank t...,0.130328,0.496618
2,148,Adam DeVine,"June 22, 2019",Best Time of Our Lives (2019) – Full Transcript,Frenetic comic Adam Devine talks teen awkwardn...,"[rock music playing] [indistinct chatter] Hey,...",hey thank lady gentlemen adam devine hey every...,0.030117,0.543354
3,177,Latin History for Morons,"January 27, 2019",John Leguizamo’s Road to Broadway (2018) – Tra...,Latin History for Morons: John Leguizamo's Roa...,There are people out there trying to rewrite o...,try rewrite history dangerous anything legitim...,0.023682,0.495872
4,156,Colin Quinn,"May 16, 2019",The New York Story (2016) – Full Transcript,Colin Quinn discusses the origins of New York ...,[disco music playing] [horn blows] ♪ Young and...,young pretty new york city girl twenty five he...,0.030559,0.418339
5,340,Jim Jefferies,"September 7, 2017",Alcoholocaust (2010) – Transcript,*Alcoholocaust: (Meaning: The aftermath of a d...,[Horn honks] [Indistinct conversations] [scatt...,lady gentleman cue start clap start cheer star...,0.096071,0.497073
6,246,NaN,"February 22, 2018",Eddie Izzard – Glorious (1997) – Transcript,Eddie Izzard's routine has a loose trajectory ...,"(Siren) Well, Times Square. I just had to do a...",square show brilliant square always bunch hang...,0.026441,0.491677
7,252,EDDIE IZZARD,"February 8, 2018",STRIPPED (2009) – Full Transcript,The acclaimed comedian of the surreal performs...,(AUDIENCE CHEERING) London! London! London. Lo...,london london london london london world excep...,0.021785,0.494845
8,24,Mike Epps,"May 26, 2022",Don’t Take It Personal (2015) | Transcript,Mike Epps tears up the stage at L.A.'s histori...,[upbeat piano playing] [piano crescendos] [ind...,lady gentleman dj brandi garcia give miss chlo...,0.025923,0.445427
9,9,Fortune Feimster,"November 28, 2022",Good Fortune (2022) | Transcript,"Fortune reflects on her own good fortune, incl...",[upbeat music plays] [audience cheering] [anno...,please welcome fortune feimster powerful woman...,0.105192,0.497838


In [7]:
#find the rows in 'valid_transcripts_sent_df' whose title is in 'filtered_transcripts_df' 
filtered_transcripts_sent_df = valid_transcripts_sent_df[valid_transcripts_sent_df['Title'].isin(filtered_transcripts_df['Title'])]
filtered_transcripts_sent_df

,Comedian,Title,Subtitle,Transcript Part,Action,Processed Transcript,Audience Reaction,Funniness
39,Fortune Feimster,Good Fortune (2022) | Transcript,"Fortune reflects on her own good fortune, incl...",♪ You know we like to keep it lowkey ♪ ♪ Wind...,NA,keep lowkey window feel breeze care worry try ...,NA,0
1011,Mike Epps,Don’t Take It Personal (2015) | Transcript,Mike Epps tears up the stage at L.A.'s histori...,We got all kinds of ages in here tonight. You...,[audience laughing],kind age tonight house baby baby bedtime n gga...,laugh,2
1012,Mike Epps,Don’t Take It Personal (2015) | Transcript,Mike Epps tears up the stage at L.A.'s histori...,Where the ’90 babies at? ’90?,[audience cheering],baby,cheer,1
1013,Mike Epps,Don’t Take It Personal (2015) | Transcript,Mike Epps tears up the stage at L.A.'s histori...,“Mama… Fat-ass b*tch! Fuck!” That’s when your...,[audience cheering],mama… fat b tch brother room whoo wee tore b t...,cheer,1
1014,Mike Epps,Don’t Take It Personal (2015) | Transcript,Mike Epps tears up the stage at L.A.'s histori...,You can borrow money from a white girl and do...,[audience laughing],borrow money white girl never give last tyrone...,laugh,2
...,...,...,...,...,...,...,...,...
4604,NaN,Eddie Izzard – Glorious (1997) – Transcript,Eddie Izzard's routine has a loose trajectory ...,‘You can’t fight off the speed of the life he...,(Audience cheer),‘ fight speed life life leave lone moody moody...,cheer,1
4605,NaN,Eddie Izzard – Glorious (1997) – Transcript,Eddie Izzard's routine has a loose trajectory ...,“‘A problem of type 2094 has occurred.’ “What...,NA,‘ problem type occur problem miss control p pr...,NA,0
4662,EDDIE IZZARD,STRIPPED (2009) – Full Transcript,The acclaimed comedian of the surreal performs...,"”And Mr GeraId, the giant squid.” ”Giant squi...",NA,mr geraid giant squid giant squid diary day mo...,NA,0
5835,Jim Jefferies,Alcoholocaust (2010) – Transcript,*Alcoholocaust: (Meaning: The aftermath of a d...,"and everything stopped. And she walked in, an...",NA,everything stop walk well…it andrew unison wen...,NA,0


In [8]:
#get model that is pre-trained on all valid transcripts, and also the vectorizer, and features
full_grad_clf = pickle.load(open('trained_grad_model', 'rb'))
full_vectorizer = pickle.load(open('full_vectorizer', 'rb'))
full_features = pickle.load(open('full_features', 'rb'))

#TF-IDF vectorization on 'filtered_transcripts_sent_df' using full_vectorizer
filtered_tfidf_matrix = full_vectorizer.transform(filtered_transcripts_sent_df['Processed Transcript'])

X_train_filtered = pd.DataFrame(filtered_tfidf_matrix.toarray(), columns = full_vectorizer.get_feature_names_out())
X_train_filtered = X_train_filtered[full_features]
y_train_filtered = filtered_transcripts_sent_df['Funniness']

/var/folders/r6/8w9yhb292k5_7hy_mpvj9p780000gn/T/ipykernel_14652/3617737525.py:3: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  full_vectorizer = pickle.load(open('full_vectorizer', 'rb'))


In [9]:
#get counts of each funniness in 'y_train_filtered'
y_train_filtered.value_counts()

2    152
1     18
0      9
Name: Funniness, dtype: int64

In [10]:
#oversample to account for inbalanced data
oversampler = RandomOverSampler(sampling_strategy = 'minority')
X_train_sm, y_train_sm = oversampler.fit_resample(X_train_filtered, y_train_filtered)
y_train_sm.value_counts()

0    152
2    152
1     18
Name: Funniness, dtype: int64

In [11]:
#train gradient boosting classifier model on X_train_sm, y_train_sm
from sklearn.ensemble import GradientBoostingClassifier

filtered_grad_clf = GradientBoostingClassifier(n_estimators = 40, learning_rate = 0.1, random_state = 0).fit(X_train_sm, y_train_sm)

In [12]:
#function to get predictions weighted by 'full_grad_clf' and 'filtered_grad_clf'
def weighted_predictions(full_model, filtered_model, X, weight = 0.75):
    full_probs = full_model.predict_proba(X)
    filtered_probs = filtered_model.predict_proba(X)

    #determine if filtered_model has fitted to all classes
    unfitted_classes = []
    for i in range(4):
        if i not in filtered_model.classes_:
            unfitted_classes.append(i)

    #add column of zero to filtered_probs for each unfitted class
    if len(unfitted_classes) > 0:
        for c in unfitted_classes:
            filtered_probs = np.insert(filtered_probs, c, 0, axis = 1)

    adj_probs = (full_probs * weight) + (filtered_probs * (1 - weight))
    y_hat = adj_probs.argmax(axis = 1)
    return y_hat

In [13]:
#break 'user_input' down into 10 chunks
def break_chunks(input, n_chunks):
    user_tokens = [s for s in input]
    user_token_count = len(user_tokens)
    chunk_token_count = user_token_count // n_chunks
    chunk_ranges = range(0, user_token_count, chunk_token_count)
    chunks = []

    for c in range(n_chunks):
        if c == n_chunks - 1:
            chunk = ''.join(user_tokens[chunk_ranges[c]:])
        else:
            chunk = ''.join(user_tokens[chunk_ranges[c]:chunk_ranges[c + 1]])
        chunks.append(chunk)

    return chunks

user_chunks = break_chunks(user_input, 10) 

In [14]:
#preprocess each chunk in 'user_chunks' and use the 'full_vectorizer' to transform it
chunk_df = pd.DataFrame({'Chunk': user_chunks})
chunk_df['Processed Chunk'] = chunk_df['Chunk'].apply(preprocess)
vec_user_input = full_vectorizer.transform(chunk_df['Processed Chunk'])
user_input_df = pd.DataFrame(vec_user_input.toarray(), columns = full_vectorizer.get_feature_names_out())
user_input_df = user_input_df[full_features]

#make predictions of funniness of 'user_input' 
'''
warm_pred_prob = full_grad_clf.predict_proba(user_input_df)
warm_pred = np.argmax(warm_pred_prob, axis = 1)
'''
weighted_pred = weighted_predictions(full_grad_clf, filtered_grad_clf, user_input_df)

#add column for predictions of funniness
chunk_df['Funniness'] = weighted_pred
chunk_df

,Chunk,Processed Chunk,Funniness
0,Join me in welcoming the author of six number ...,join welcome author six number new york bestse...,3
1,"ed.” I’m like, “You think I’m gonna ruin a buz...",ed ruin buzz defecate never happen sat trip ac...,0
2,"aid, “You need to set an intention.” I was lik...",aid need set intention care set need set less ...,0
3,y a man would see a plant and be like… Only a ...,plant like… woman ever history world woman cac...,2
4,"he security camera…” Always, I always say, ‘He...",security camera… always always ‘ hey chelsea h...,0
5,quick.” I’m in a denim onesie right now. I ha...,quick denim onesie orthotics every pair shoe e...,2
6,rst panic attack. I don’t know what else you c...,rst panic attack else could describe like… sud...,2
7,with a frozen ham underneath her arm. And I’m ...,frozen ham underneath arm cook ham open freeze...,2
8,esh air blowing in my face and be topless. He ...,esh air blow face topless worry meet two week ...,2
9,I never thought I would use. But it made sens...,never use sense put piece together grateful si...,0


In [15]:
#function to output funniness prediction in word form
def get_pred_output(pred):
    if pred == 0:
        return 'Neutral'
    elif pred == 1:
        return 'A little funny'
    elif pred == 2:
        return 'Moderately funny'
    elif pred == 3:
        return 'Very funny'

chunk_df['Funniness Word'] = chunk_df['Funniness'].apply(get_pred_output)
chunk_df

,Chunk,Processed Chunk,Funniness,Funniness Word
0,Join me in welcoming the author of six number ...,join welcome author six number new york bestse...,3,Very funny
1,"ed.” I’m like, “You think I’m gonna ruin a buz...",ed ruin buzz defecate never happen sat trip ac...,0,Neutral
2,"aid, “You need to set an intention.” I was lik...",aid need set intention care set need set less ...,0,Neutral
3,y a man would see a plant and be like… Only a ...,plant like… woman ever history world woman cac...,2,Moderately funny
4,"he security camera…” Always, I always say, ‘He...",security camera… always always ‘ hey chelsea h...,0,Neutral
5,quick.” I’m in a denim onesie right now. I ha...,quick denim onesie orthotics every pair shoe e...,2,Moderately funny
6,rst panic attack. I don’t know what else you c...,rst panic attack else could describe like… sud...,2,Moderately funny
7,with a frozen ham underneath her arm. And I’m ...,frozen ham underneath arm cook ham open freeze...,2,Moderately funny
8,esh air blowing in my face and be topless. He ...,esh air blow face topless worry meet two week ...,2,Moderately funny
9,I never thought I would use. But it made sens...,never use sense put piece together grateful si...,0,Neutral


In [16]:
#topic modelling of similar transcripts
from empath import Empath
lexicon = Empath()

def get_topics(text, n_topics):
    topics_lst = []
    top_dict = lexicon.analyze(text, normalize = True)
    sorted_top_lst = sorted(top_dict.items(), key = lambda x: x[1], reverse = True)

    for top_weight in sorted_top_lst[:n_topics]:
        topics_lst.append(top_weight[0])
    return topics_lst

filtered_transcripts_df['Topics'] = filtered_transcripts_df['Cleaned Transcript'].apply(get_topics, args = (3,))
filtered_transcripts_df

,Unnamed: 0,Comedian,Date,Title,Subtitle,Transcript,Cleaned Transcript,Polarity,Subjectivity,Topics
0,391,Patrice O’Neal,"April 28, 2017",Elephant in the Room (2011) – Transcript,"Transcript of 'Elephant in the Room', stand up...",[Light funk music] * [cheers and applause] Tha...,thank thank thank thank thank thank particular...,0.029343,0.532460,"[negative_emotion, children, violence]"
1,117,Jim Gaffigan,"January 19, 2020",Quality Time (2019) – Full Transcript,Comedian Jim Gaffigan performs live at the Sta...,"(audience cheering, applauding) Thank you! Tha...",audience cheer applaud thank thank god thank t...,0.130328,0.496618,"[animal, pet, negative_emotion]"
2,148,Adam DeVine,"June 22, 2019",Best Time of Our Lives (2019) – Full Transcript,Frenetic comic Adam Devine talks teen awkwardn...,"[rock music playing] [indistinct chatter] Hey,...",hey thank lady gentlemen adam devine hey every...,0.030117,0.543354,"[negative_emotion, children, friends]"
3,177,Latin History for Morons,"January 27, 2019",John Leguizamo’s Road to Broadway (2018) – Tra...,Latin History for Morons: John Leguizamo's Roa...,There are people out there trying to rewrite o...,try rewrite history dangerous anything legitim...,0.023682,0.495872,"[family, children, negative_emotion]"
4,156,Colin Quinn,"May 16, 2019",The New York Story (2016) – Full Transcript,Colin Quinn discusses the origins of New York ...,[disco music playing] [horn blows] ♪ Young and...,young pretty new york city girl twenty five he...,0.030559,0.418339,"[negative_emotion, children, speaking]"
5,340,Jim Jefferies,"September 7, 2017",Alcoholocaust (2010) – Transcript,*Alcoholocaust: (Meaning: The aftermath of a d...,[Horn honks] [Indistinct conversations] [scatt...,lady gentleman cue start clap start cheer star...,0.096071,0.497073,"[negative_emotion, friends, children]"
6,246,NaN,"February 22, 2018",Eddie Izzard – Glorious (1997) – Transcript,Eddie Izzard's routine has a loose trajectory ...,"(Siren) Well, Times Square. I just had to do a...",square show brilliant square always bunch hang...,0.026441,0.491677,"[negative_emotion, movement, animal]"
7,252,EDDIE IZZARD,"February 8, 2018",STRIPPED (2009) – Full Transcript,The acclaimed comedian of the surreal performs...,(AUDIENCE CHEERING) London! London! London. Lo...,london london london london london world excep...,0.021785,0.494845,"[animal, negative_emotion, movement]"
8,24,Mike Epps,"May 26, 2022",Don’t Take It Personal (2015) | Transcript,Mike Epps tears up the stage at L.A.'s histori...,[upbeat piano playing] [piano crescendos] [ind...,lady gentleman dj brandi garcia give miss chlo...,0.025923,0.445427,"[children, negative_emotion, family]"
9,9,Fortune Feimster,"November 28, 2022",Good Fortune (2022) | Transcript,"Fortune reflects on her own good fortune, incl...",[upbeat music plays] [audience cheering] [anno...,please welcome fortune feimster powerful woman...,0.105192,0.497838,"[wedding, party, home]"


In [17]:
#topic modelling of user input chunks
chunk_df['Topics'] = chunk_df['Processed Chunk'].apply(get_topics, args = (3,))
chunk_df

,Chunk,Processed Chunk,Funniness,Funniness Word,Topics
0,Join me in welcoming the author of six number ...,join welcome author six number new york bestse...,3,Very funny,"[speaking, music, communication]"
1,"ed.” I’m like, “You think I’m gonna ruin a buz...",ed ruin buzz defecate never happen sat trip ac...,0,Neutral,"[traveling, communication, air_travel]"
2,"aid, “You need to set an intention.” I was lik...",aid need set intention care set need set less ...,0,Neutral,"[negative_emotion, movement, giving]"
3,y a man would see a plant and be like… Only a ...,plant like… woman ever history world woman cac...,2,Moderately funny,"[children, eating, restaurant]"
4,"he security camera…” Always, I always say, ‘He...",security camera… always always ‘ hey chelsea h...,0,Neutral,"[love, positive_emotion, sadness]"
5,quick.” I’m in a denim onesie right now. I ha...,quick denim onesie orthotics every pair shoe e...,2,Moderately funny,"[body, swimming, sexual]"
6,rst panic attack. I don’t know what else you c...,rst panic attack else could describe like… sud...,2,Moderately funny,"[party, negative_emotion, positive_emotion]"
7,with a frozen ham underneath her arm. And I’m ...,frozen ham underneath arm cook ham open freeze...,2,Moderately funny,"[family, children, home]"
8,esh air blowing in my face and be topless. He ...,esh air blow face topless worry meet two week ...,2,Moderately funny,"[negative_emotion, children, family]"
9,I never thought I would use. But it made sens...,never use sense put piece together grateful si...,0,Neutral,"[family, air_travel, traveling]"


In [18]:
#topic modelling of entire user input
user_topics = get_topics(preprocess(user_input), 3)
user_topics

['children', 'negative_emotion', 'family']